In [ ]:
# !pip install numpy
# !pip install pandas
# !pip install glob2
# !pip install time
# !pip install catboost
# !pip install sklearn
# !pip install pickle

In [26]:
path = input('Введите путь к папке с исходными файлами')

Введите путь к папке с исходными файламиC:\Users\shers\Desktop\SF\Буровая\test


In [27]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
import pickle
import itertools
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import roc_auc_score, f1_score, silhouette_score, homogeneity_score, completeness_score, v_measure_score, confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

In [53]:
%load_ext autoreload
%autoreload 1
%aimport reading_data
reading_data.walk_dir(path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
data = pd.read_csv('data.csv')
data.shape

(234957, 135)

In [55]:
X =data[[
 'DEPT', 'HDTH', 'BPOS', 'HKLD', 'STOR', 'FLWI', 'RPM',
 'SPPA', 'ECD', 'DLS', 'INCL', 'AZIM', 'GR', 'APRS', 'BVEL', 'RIG_STATE',
 'ESD', 'Stick', 'DDEPT_1', 'DDEPT_3', 'DDEPT',
 'DDEPT_6', 'FDEPT_1', 'DDEPT_12', 'DDEPT_18',
 'FDEPT_3', 'DBPOS', 'DBPOS_1', 'DBPOS_3',
 'DBPOS_6', 'FBPOS_1', 'DBPOS_12', 'DBPOS_18','FBPOS_3', 
 'DHKLD', 'DHKLD_1', 'DHKLD_3', 'DHKLD_6', 'DHKLD_12', 'DHKLD_18',
 'FHKLD_1', 'FHKLD_3', 'DSPPA', 'DSPPA_1',
 'DSPPA_3', 'DSPPA_6', 'DSPPA_12', 'DSPPA_18',
 'FSPPA_1', 'FSPPA_3', 'DRPM', 'DRPM_1', 'DRPM_3', 'DRPM_6', 'DRPM_12',
 'FRPM_1', 'FRPM_3', 'DRPM_18', 
 'DBVEL', 'DBVEL_1', 'DBVEL_3', 'DBVEL_6', 'FBVEL_1', 'DBVEL_12', 'DBVEL_18',
 'FBVEL_3', 'DSTOR_1', 'DSTOR_3', 'FSTOR_1', 'FSTOR_3',
 'DSTOR_6', 'DSTOR_12', 'DSTOR_18',
 'SPPA_APRS', 'DSPPA_APRS_1', 'DSPPA_APRS_3', 'DSPPA_APRS_6', 'FSPPA_APRS_1',
 'DSPPA_APRS_12', 'DSPPA_APRS_18',
 'FSPPA_APRS_3', 'DECD', 'ANOMAL', #'TimeU',
 'discardSPPA', 'discardSTOR', 'discardECD', 'discardStick', 'discardAPRS', 'discardRPM',
 'PDEPT', 'F', 'F_all', 'conner', 'conner_delta', 'GR_type',
 'DEPT_bin', 'SPPA_bin', 'SPPA_APRS_bin', 'BVEL_bin', 'HKLD_bin', 'BPOS_bin',
 'RPM_bin', 'STOR_bin', 'DEPT_bin_new', 'SPPA_bin_new', 'SPPA_APRS_bin_new',
 'BVEL_bin_new', 'HKLD_bin_new', 'BPOS_bin_new', 
 'RPM_bin_new', 'STOR_bin_new', 'DEPT_back3', 'SPPA_back3', 'SPPA_APRS_back3',
 'BVEL_back3', 'HKLD_back3', 'BPOS_back3', 
 'RPM_back3', 'STOR_back3', 'DEPT_back1', 'SPPA_back1', 'SPPA_APRS_back1',
 'BVEL_back1', 'HKLD_back1', 'BPOS_back1', 
 'RPM_back1', 'STOR_back1'
#     'F', 'STOR', 'SPPA_APRS', 'FSTOR_3', 'HKLD_bin_new', #'TimeU', 
#     'DSPPA_3', 'BVEL_bin', 'DSPPA_18', 'DSPPA_6', 'FDEPT_3', 'ANOMAL', #'DBPOS_6', 
#     'DBVEL_6', 'HKLD', 'DBVEL_12', 'DHKLD_12', 'DBVEL_18', #'FBPOS_3', 
#     'DRPM', 'RPM',  'FLWI',
#     'SPPA_APRS_back1', 'ESD', 'FSPPA_3', 'DSTOR_18', 'DSPPA_12', #'BPOS', 'DBPOS_18', 
#     'FHKLD_3', 'DSPPA_APRS_3', 'DBVEL_3', 'DDEPT_18', 'DRPM_18', 'ECD', 'PDEPT', 'DSTOR_12', 
#     'FSPPA_APRS_3', 'SPPA', 'DHKLD_18', 'DHKLD_6', 'DSPPA', 'DEPT', 'F_all', #'DBPOS_12'
]].values
# нормализация:
sc= StandardScaler().fit(X)
X = sc.transform(X)

In [56]:
with open('ctboost_model_0706.pkl', 'rb') as pkl_file:
    catboost_model = pickle.load(pkl_file)

predict = catboost_model.predict_proba(X)
# predict = catboost_model.predict(X)
# predict_list = predict.tolist()

In [57]:
df=pd.DataFrame()
df = pd.DataFrame((_ for _ in itertools.zip_longest(*predict))).T
df.rename(columns = {0:'predict', 1:'Predict'}, inplace = True)
df['mark'] = 0
df.loc[df['Predict'] > 0.97, 'mark'] = 1
predict_list = df['mark'].tolist()
display(df)

,predict,Predict,mark
0,0.856743,0.143257,0
1,0.858173,0.141827,0
2,0.866333,0.133667,0
3,0.867846,0.132154,0
4,0.860475,0.139525,0
...,...,...,...
234952,0.376860,0.623140,0
234953,0.369423,0.630577,0
234954,0.295899,0.704101,0
234955,0.359940,0.640060,0


In [58]:
#Удаление одиноких единиц и нолей
for i in range(len(predict_list)-4):
#     if predict_list[i]==0 and predict_list[i-1]==1 and predict_list[i+1]==1:
#         predict_list[i]=1
#     if predict_list[i]==0 and predict_list[i-1]==1 and predict_list[i+2]==1:
#         predict_list[i]=1
    
    if predict_list[i]==1 and predict_list[i-1]==0 and predict_list[i+1]==0:
        predict_list[i]=0
    if predict_list[i]==1 and predict_list[i-1]==0 and predict_list[i+2]==0:
        predict_list[i]=0
    if predict_list[i]==1 and predict_list[i-1]==0 and predict_list[i+3]==0:
        predict_list[i]=0
    if predict_list[i]==1 and predict_list[i-1]==0 and predict_list[i+4]==0:
        predict_list[i]=0

In [59]:
%load_ext autoreload
%aimport primary_data
primary_data.walk_dir(path)
predict_data = pd.read_csv('primary_data.csv')
# predict_data['Predict'] = df['Predict']*100
predict_data['Predict'] = np.array(predict_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#Формирование уведомления о низком качестве входных данных
# predict_data.loc[((predict_data['ECD'].rolling(20).mean()==(-999.25)) |
#                   (predict_data['RPM'].rolling(20).mean()==(-999.25)) |
#                   (predict_data['BVEL'].rolling(20).mean()==(-999.25)) |
#                   (predict_data['HKLD'].rolling(20).mean()==(-999.25)) | 
#                   (predict_data['SPPA'].rolling(20).mean()==(-999.25)) | 
#                   (predict_data['STOR'].rolling(20).mean()==(-999.25)) | 
#                   (predict_data['BPOS'].rolling(20).mean()==(-999.25))), 'Predict'] = 'Плохие данные'

In [60]:
predict_data.to_excel('final_data_test_0706_1.xlsx')

In [7]:
#Оценка метрик по размеченным данным
labels=["Норма","Прихват"]
predict_data = predict_data.fillna(0)
print(confusion_matrix(predict_data['Predict'].values, predict_data['StuckPipe'].values))  
print(classification_report(predict_data['Predict'].values, predict_data['StuckPipe'].values,
                            target_names = labels)) 

[[30928    70]
 [  183  1517]]
              precision    recall  f1-score   support

       Норма       0.99      1.00      1.00     30998
     Прихват       0.96      0.89      0.92      1700

    accuracy                           0.99     32698
   macro avg       0.98      0.95      0.96     32698
weighted avg       0.99      0.99      0.99     32698



In [17]:
predict_data.Predict.value_counts()

0.0    14841
1.0    11669
Name: Predict, dtype: int64

In [52]:
predict_data.marker.value_counts()

AttributeError: 'DataFrame' object has no attribute 'marker'